In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading Data**

Importing and storing text files in a string

In [2]:
with open('/content/drive/MyDrive/data.txt','r') as f:
  text = f.readlines()

print(len(text))
data=[]
data+=text
text[:3]

4652818


['sai kha ya her kisi kay bus ki bat nhi hai lakin main ki hal kal bi aj aur aj bi sirf aus say bus\n',
 'kya bt hai\n',
 'wah je wah\n']

In [3]:
 misspellings=[]
 with open('/content/drive/MyDrive/misspellings.txt','r') as file:  
  for line in file:
    sen=[]      
    for word in line.split():
      word=word.replace(",", "").strip("\n").lower()
      sen.append(word)  
    misspellings.append(sen)  
misspellings=misspellings[1:]
print(len(misspellings))
misspellings[:3]

36100


[['ka', 'kaz', 'cka', 'mka', 'kga', 'yka', 'kba'],
 ['hai', 'qhai', 'haoi', 'ghai', 'haiu', 'hmi', 'qai', 'yhai'],
 ['mein', 'mvin', 'mqein', 'meint', 'mevn']]

Tokenization of sentences to create a vocabulary of words

In [4]:
#-- Removing newline character from every sentence and changing upercase words to lower
for i in range(0,len(data)):
  data[i]=data[i].strip("\n").lower()

In [5]:
data[0],text[0]

('sai kha ya her kisi kay bus ki bat nhi hai lakin main ki hal kal bi aj aur aj bi sirf aus say bus',
 'sai kha ya her kisi kay bus ki bat nhi hai lakin main ki hal kal bi aj aur aj bi sirf aus say bus\n')

In [6]:
#-- Tokenization of sentences
vocab=[]
x=0
for sentence in data:
  words=sentence.split()
  vocab+=words
# vocab

# **Language Model  - P(w)**

Generating Word Uigrams:
   We have a ditionary to store unigram as key and its count as value.

In [7]:
#-  Function returns unigram of words
def unigramWords(vocab):
  diction= {}
  for i in range(len(vocab)):
    wrd = str(vocab[i])
    # already exists
    if (wrd in diction):
      diction[wrd]+=1 
    else:
      diction[wrd] = 1   #words doesn't exist
  return diction


Generating Character Uigrams:
   We have a ditionary to store unigram as key and its count as value.

In [8]:
#-  Function returns unigram of chaarcters
def unigramChars(vocab):
  letters    = 'abcdefghijklmnopqrstuvwxyz'
  diction={x:0 for x in letters}
  diction
  for wrd in vocab:
    for c in wrd:
      if (c in diction):
        diction[c]+=1 
  return diction

Generating Character Bigrams:
   We have a ditionary to store first char as key,which corespond to a sub-dictionary where key is second char and value is their count.

In [9]:

def bigramChars(vocab):
  letters    = 'abcdefghijklmnopqrstuvwxyz'
  diction = {outer_letter: {inner_letter: 0 for inner_letter  in letters} for outer_letter  in letters}
  for wrd in vocab:
    for i in range(len(wrd)-1):
      if (wrd[i] in diction):
        diction[ wrd[i] ][ wrd[i+1] ]+=1 
  return diction


In [10]:
biChars=bigramChars(vocab)  ## took 1m 46s  to ecexute

In [11]:
uniW = unigramWords(vocab) # took around 25 sec to execute

In [12]:
uniC = unigramChars(vocab)  # took around 45 sec to execute

In [13]:
#-- our new vocabulary will be a unique list of words, previous 'vocab' contains duplicates
Vocabulary=uniW.keys()

In [14]:
Nc=sum(uniC.values())  #total words
# Nb=sum(biChars.values())  #total words
Nw=len(vocab)

# for k, v in uniC.items():
#     uniC[k] = v /Nc
# for k, v in uniW.items():
#     uniW[k] = v /Nw
list(uniC.items())[:2] , list(biChars.items())[:1]

([('a', 61215602), ('b', 6203369)],
 [('a',
   {'a': 5471533,
    'b': 1532556,
    'c': 377675,
    'd': 1225206,
    'e': 85558,
    'f': 527638,
    'g': 949703,
    'h': 3323165,
    'i': 4368123,
    'j': 346544,
    'k': 1493423,
    'l': 2116667,
    'm': 1849657,
    'n': 4277763,
    'o': 114394,
    'p': 980441,
    'q': 679384,
    'r': 8003786,
    's': 1405662,
    't': 2762745,
    'u': 1510076,
    'v': 46752,
    'w': 483862,
    'x': 6625,
    'y': 3152455,
    'z': 1018428})])

#  **Confusion Matrices**

In [16]:
letters    = 'abcdefghijklmnopqrstuvwxyz@'
Insert = {outer_letter: {inner_letter: 0 for inner_letter  in letters} for outer_letter  in letrs}  #includes @ as first char
Delete = {outer_letter: {inner_letter: 0 for inner_letter  in letters} for outer_letter  in letrs}  #includes @ as first char
Transpose  = {outer_letter: {inner_letter: 0 for inner_letter  in letters} for outer_letter  in letters}
Substitute  = {outer_letter: {inner_letter: 0 for inner_letter  in letters} for outer_letter  in letters}

In [18]:
def check_delete(typo,correct):
  for i in range(len(typo)):
    wrd=typo[:i] + typo[i+1:]
    if (wrd==correct):
      if i ==0:
        prev='@'
      else:
         prev=wrd[i-1] 
      return i,prev,typo[i]
  return None,None,None

def check_insert(typo,correct):
  letters    = 'abcdefghijklmnopqrstuvwxyz'
  for i in range(len(typo)+1):
    for c in letters:
      wrd=typo[:i] +c+ typo[i:]#add char
      if (wrd==correct):
        if i ==0:
          prev='@'
        else:
          prev=wrd[i-1] 
        return i,prev,wrd[i]
  return None,None,None

def check_substitute(typo,correct):
  letters    = 'abcdefghijklmnopqrstuvwxyz'
  for i in range(len(typo)+1):
    for c in letters:
      wrd=typo[:i] +c+ typo[i+1:] #replace character
      if (wrd==correct):
        return i,typo[i],wrd[i]
  return None,None,None

def check_transpose(typo,correct):
  for i in range(len(typo)-1):
    twoChars=typo[i:i+2]
    wrd = typo[:i] + twoChars[::-1] + typo[i+2:]
    if (wrd==correct):
      return i,twoChars[0],twoChars[1]
  return None,None,None

In [20]:
#   Populating dictionaries of insert delete ...
for m in misspellings:

  for i in range(1,len(m)):
      #typo,correct
    idx,prev_char,del_char=check_delete(m[i],m[0])   #(0, '@', 't')
    if (idx!=None):
      # print('D ',del_char,prev_char,i,end=',   ')
      Delete[prev_char][del_char]+=1 
      continue

    idx,prev_char,ins_char=check_insert(m[i],m[0])
    if (idx!=None):
      Insert[prev_char][ins_char]+=1
      continue

    idx,X,Y=check_substitute(m[i],m[0]) #X -> Y   (2, 't', 'r')
    if (idx!=None):
      Substitute[X][Y]+=1
      continue

    idx,X,Y=check_transpose(m[i],m[0])  #XY -> YX 
    if (idx!=None):
      Transpose[X][Y]+=1 
      continue


In [21]:
print(Insert)

{'a': {'a': 0, 'b': 9, 'c': 8, 'd': 16, 'e': 2, 'f': 3, 'g': 10, 'h': 15, 'i': 11, 'j': 3, 'k': 17, 'l': 26, 'm': 21, 'n': 30, 'o': 0, 'p': 2, 'q': 2, 'r': 40, 's': 10, 't': 40, 'u': 0, 'v': 3, 'w': 4, 'x': 1, 'y': 12, 'z': 6, '@': 0}, 'b': {'a': 24, 'b': 0, 'c': 0, 'd': 1, 'e': 6, 'f': 1, 'g': 0, 'h': 5, 'i': 6, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 2, 'p': 0, 'q': 0, 'r': 5, 's': 0, 't': 0, 'u': 6, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 3, '@': 0}, 'c': {'a': 5, 'b': 0, 'c': 0, 'd': 0, 'e': 6, 'f': 0, 'g': 0, 'h': 21, 'i': 2, 'j': 0, 'k': 3, 'l': 2, 'm': 0, 'n': 0, 'o': 7, 'p': 0, 'q': 0, 'r': 2, 's': 1, 't': 4, 'u': 1, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 0, '@': 0}, 'd': {'a': 20, 'b': 0, 'c': 0, 'd': 0, 'e': 15, 'f': 0, 'g': 2, 'h': 4, 'i': 13, 'j': 0, 'k': 0, 'l': 0, 'm': 1, 'n': 2, 'o': 7, 'p': 0, 'q': 0, 'r': 0, 's': 1, 't': 0, 'u': 4, 'v': 0, 'w': 1, 'x': 0, 'y': 1, 'z': 0, '@': 0}, 'e': {'a': 6, 'b': 1, 'c': 8, 'd': 12, 'e': 0, 'f': 1, 'g': 3, 'h': 6, 'i': 3, 'j': 1, 'k'

# **Candidate Words Generation**


  Stores candidates in a dictionary where key is candidate word and first value of tuple is operation Insert=I, Delete=D, Substitue=S, Transpose=T
  eg. candidates of acress
  {'actress': ('D', 't', 2), 'acres': ('I', 's', 4), 'caress': ('T', 'ca','ac'0), 'access': ('S', 'r', 'c', 2)}

In [25]:
def getCandidates(typo,voc):
  short_vocab=[x for x in voc if (abs(len(x)-len(typo))<=1)]
  candidates={}
  for c in short_vocab:
    idx,prev_char,del_char=check_delete(typo,c)   #(0, '@', 't')
    if (idx!=None):
      candidates[c]=('D',prev_char,del_char,idx) 
      continue

    idx,prev_char,ins_char=check_insert(typo,c)
    if (idx!=None):
      candidates[c]=('I',prev_char,ins_char,idx) 
      continue

    idx,X,Y=check_substitute(typo,c) #X -> Y   (2, 't', 'r')
    if (idx!=None):
      candidates[c]=('S',X,Y,idx) 
      continue

    idx,X,Y=check_transpose(typo,c)  #XY -> YX 
    if (idx!=None):
      candidates[c]=('t',X+Y,Y+X,idx)  
    
  return candidates

In [26]:
candidates=getCandidates("ker",Vocabulary)
print(candidates)

{'her': ('S', 'k', 'h', 0), 'kar': ('S', 'e', 'a', 1), 'ke': ('D', 'e', 'r', 2), 'key': ('S', 'r', 'y', 2), 'keh': ('S', 'r', 'h', 2), 'kr': ('D', 'k', 'e', 1), 'ker': ('S', 'k', 'k', 0), 'per': ('S', 'k', 'p', 0), 'kea': ('S', 'r', 'a', 2), 'kei': ('S', 'r', 'i', 2), 'kher': ('I', 'k', 'h', 1), 'kre': ('t', 'er', 're', 1), 'kero': ('I', 'r', 'o', 3), 'kee': ('S', 'r', 'e', 2), 'der': ('S', 'k', 'd', 0), 'krr': ('S', 'e', 'r', 1), 'kdr': ('S', 'e', 'd', 1), 'fer': ('S', 'k', 'f', 0), 'kur': ('S', 'e', 'u', 1), 'kere': ('I', 'r', 'e', 3), 'eer': ('S', 'k', 'e', 0), 'er': ('D', '@', 'k', 0), 'ver': ('S', 'k', 'v', 0), 'yer': ('S', 'k', 'y', 0), 'oker': ('I', '@', 'o', 0), 'ber': ('S', 'k', 'b', 0), 'ser': ('S', 'k', 's', 0), 'ger': ('S', 'k', 'g', 0), 'kor': ('S', 'e', 'o', 1), 'ler': ('S', 'k', 'l', 0), 'mer': ('S', 'k', 'm', 0), 'kera': ('I', 'r', 'a', 3), 'keu': ('S', 'r', 'u', 2), 'keo': ('S', 'r', 'o', 2), 'ter': ('S', 'k', 't', 0), 'kery': ('I', 'r', 'y', 3), 'kir': ('S', 'e', 'i',

# **Error Model P(x | w )**

In [ ]:
EM_words=[]
EM_probs=[]
for wrd in candidates:
  EM_words.append(wrd)
  if candidates[wrd][0]=='S':
    t=candidates[wrd][1]
    c=candidates[wrd][2]
    Substitute[t][c]/uniC[c]



# **Selection Model using argmax**

In [ ]:
import numpy as np
 
probs = np.array(EM_probs)
wrds= np.array(EM_words)
i=np.argmax(probs)
CorrectedWord=wrds[i]
print(CorrectedWord)